<a href="https://colab.research.google.com/github/cobaltyang/ChatGPT-Web/blob/main/yuantpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install keras_preprocessing


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.0 MB/s eta 0:00:00


In [10]:
import os
import tensorflow as tf

try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
  print('TPU not found')



Found TPU at: grpc://10.14.66.186:8470


In [31]:
# @title My Code

from google.colab import drive
drive.mount('/content/drive')
!pwd
import sys
sys.path.append('/content/drive/MyDrive/yuan/yuan')

from ghostNet import GhostNet

import numpy as np
import tensorflow as tf
import os
import random
import scipy.io as sio
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras import optimizers
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Reshape, Dropout,ReLU
import time
import matplotlib.pyplot as plt


# # 启用GPU的内存增长
# physical_devices = tf.config.list_physical_devices('GPU')
# if physical_devices:
#     try:
#         tf.config.set_logical_device_configuration(
#             physical_devices[0],
#             [tf.config.LogicalDeviceConfiguration(memory_limit=7000)])  # 可根据需要设置内存限制
#         print("GPU memory growth enabled.")
#     except RuntimeError as e:
#         print(e)
# else:
#     print("No GPU devices found.")


# tf.debugging.set_log_device_placement(True)

print(f"Tensorflow version: {tf.__version__}")

# ---------------------网络结构搭建------------------------- #
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 加载数据
print('Loading data...')

# 划分
f = '/content/drive/MyDrive/yuan/yuan/Train_set_.mat'
train_set = sio.loadmat(f)  # 导入后的数据是一个字典，取出想要的变量字段即可

dim = 16
x_train = np.concatenate(
    (train_set['R_train_real'].transpose(2, 0, 1).reshape(-1, dim, dim, 1),
     train_set['R_train_imag'].transpose(2, 0, 1).reshape(-1, dim, dim, 1)),
    axis=3)

y_train = np.concatenate(
    (train_set['w_train_real'].transpose(2, 0, 1).reshape(-1, dim),
     train_set['w_train_imag'].transpose(2, 0, 1).reshape(-1, dim)),
    axis=1)




tpu_strategy = tf.distribute.TPUStrategy(tpu_resolver)







# 数据打乱
index = [i for i in range(x_train.shape[0])]
random.shuffle(index)  # shuffle() 方法将序列的所有元素随机排序
x_train = x_train[index]
y_train = y_train[index]

# 创建模型


BATCH_SIZE = 16
# TEST_SIZE = 0.3     # ImageDataGenerator生成的验证集是前30%个
EPOCHS = 10
with tpu_strategy.scope():
  model = GhostNet((16,16,2)).build()
  # model.summary()
  tpu_optimizer = tf.keras.optimizers.Adam(lr=0.001)
# 模型编译
  model.compile(optimizer=tpu_optimizer, loss='categorical_crossentropy', metrics=['acc'])


# 模型断点保存
model_checkpoint = ModelCheckpoint('checkpoint/GhostNet-100epoch_my-datasets224_pretrainModel.hdf5', monitor='val_loss',
                                   verbose=1, save_best_only=True)

model_name = "GhostNet-my_datasets224-{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='./tmp/log/{}'.format(model_name),
                          histogram_freq=0, write_graph=True, write_images=True)

EarlyStop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto') # type: ignore
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='auto', verbose=2, min_lr=1e-10, factor=0.5, # type: ignore
                                              patience=35) # type: ignore 
checkpoint = keras.callbacks.ModelCheckpoint('my_model.h5', monitor='val_loss', verbose=0, save_best_only=True, 
                                             mode='auto') # type: ignore
# train
with tpu_strategy.scope():
  history_cnn = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=3, validation_split=0.25)
                        # callbacks=[EarlyStop, reduce_lr, checkpoint])


# 保存训练的参数
model.save_weights(f"ghostNet_my-{EPOCHS}.h5")


# -----------------------------------可视化模型训练曲线------------------------------- #
plt.figure()
history_dict = history_cnn.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
# print(history_dict.keys())
epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'b', label='Training loss')
plt.plot(epochs, val_loss_values, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
Tensorflow version: 2.12.0
Loading data...


Epoch 1/10


KeyboardInterrupt: ignored